In [ ]:
#!cp Data/sample.nc* /tmp

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
from odc.geo import geom
from odc.geo.gcp import GCPGeoBox
from tqdm.auto import tqdm

from utils.emit import emit_xr_read, gxy, sample_error

In [ ]:
%%time
xx = emit_xr_read("/tmp/sample.nc")

In [ ]:
xx.reflectance.data[xx.reflectance.data == xx.reflectance.odc.nodata] = np.nan
xx.attrs.pop("nodata", None)

In [ ]:
display(xx.odc.geobox, xx)

In [ ]:
gp, gw, pix_error, ee = sample_error(xx, 300)
#print(f"Average projection error: {pix_error.mean():.3f}px")

fig, axd = plt.subplot_mosaic(
    [
        ["A", "A", "B"],
        ["A", "A", "B"],
        ["C", "C", "C"],
    ],
    figsize=(8, 8),
)
fig.suptitle(f"Pixel Registration Error (avg:{pix_error.mean():.3f}px)")

sns.scatterplot(x=ee.T[0], y=ee.T[1], size=pix_error, ax=axd["A"])

X, Y = gxy(gp).T
sns.heatmap(
    xx.elev,
    cmap="bone",
    alpha=0.7,
    square=True,
    cbar=False,
    xticklabels=False,
    yticklabels=False,
    ax=axd["B"],
)
sns.scatterplot(
    x=X, y=Y, size=pix_error, color="y", alpha=0.3, ax=axd["B"], legend=False
)

axd["C"].axvline(pix_error.mean(), color="y")
sns.kdeplot(pix_error, ax=axd["C"])
fig.tight_layout()
pass

In [ ]:
ii = 100

ax = plt.axes(projection=ccrs.Projection("epsg:4326"))

xx.reflectance[..., ii].plot.pcolormesh(
    x="lon",
    y="lat",
    robust=True,
    cmap="magma",
    ax=ax,
    add_labels=False,
)
ax.coastlines(color='c')
pass

In [ ]:
ax = plt.axes(projection=ccrs.Projection("epsg:4326"))

xx.elev.plot.pcolormesh(
    x="lon",
    y="lat",
    robust=True,
    cmap="magma",
    add_labels=False,
)
ax.coastlines(color='c')
pass

In [ ]:
rgb = xx.reflectance.sel(band=[700, 550, 450], method='nearest')

rgb.plot.imshow(size=8, vmin=0.0, vmax=0.45, aspect=rgb.odc.geobox.aspect)

In [ ]:
import folium

mm = folium.Map()
xx.reflectance.sel(band=450, method='nearest').odc.add_to(mm, cmap="bone", robust=True)
mm.fit_bounds(xx.reflectance.odc.map_bounds())
mm

In [ ]:
plt.hist(rgb.data[..., 2].ravel(), 30)
pass

In [ ]:
xx.reflectance[..., ii].plot.imshow(
    size=8,
    robust=True,
    # imshow params
    origin="upper",
    interpolation="bicubic",
    aspect=xx.reflectance.odc.geobox.aspect,
    cmap="magma",
)

pass

In [ ]:
# xx.reflectance.plot.pcolormesh?

In [ ]:
S = 1
rois = [
    np.s_[0, ::S],
    np.s_[::S, -1],
    np.s_[-1, ::-S],
    np.s_[::-S, 0],
    np.s_[0, :1],
]


In [ ]:
pix_xy = np.dstack([xx.lon.data, xx.lat.data])
S = 10

XY = np.concatenate(
    [
        pix_xy[0, ::S],
        pix_xy[::S, -1],
        pix_xy[-1, ::-S],
        pix_xy[::-S, 0],
        pix_xy[0, :1],
    ]
)

footprint = geom.polygon(XY.tolist(), 4326).to_crs("utm")
footprint

In [ ]:
xx.lon.plot.pcolormesh(x="lon", y="lat", robust=True)

In [ ]:
gbox = xx.odc.geobox
gbox

In [ ]:
from odc.geo.geobox import GeoBox

gbox2 = GeoBox.from_bbox(gbox.extent.boundingbox, resolution=gbox.resolution)
gbox2

In [ ]:
gbox.extent.exterior | gbox2.extent.exterior

In [ ]:
%%time

yy = xx.odc.reproject(gbox2)

In [ ]:
bb = yy.reflectance.isel(band=100)
bb.where(bb != bb.odc.nodata).plot.imshow(size=6, robust=True)

In [ ]:
_ = xx.reflectance.isel(x=1000, y=750).where(xx.valid).plot.step(figsize=(12, 2))

In [ ]:
ix, iy = map(int, gbox.project(geom.point(-5.8, 36.2, "epsg:4326")).points[0])
iy, ix

In [ ]:
[xx[b].isel(x=ix, y=iy).data.item() for b in ['lon', 'lat']]

In [ ]:
%%time

xx = xx.where(xx != xx.odc.nodata)

In [ ]:
xx.chunk(band=5)

In [ ]:
plot_idx = np.arange(xx.reflectance.shape[-1])[xx.valid][::13]

xx.reflectance.isel(band=plot_idx).plot.imshow(
    col="band",
    vmin=0.1,
    vmax=0.9,
    col_wrap=4,
    size=2,
    yincrease=False,
    cmap="viridis",
    # add_labels=False,
)
pass

In [ ]:
from IPython.display import Image

Image(
    data=xx.reflectance.sel(band=550, method='nearest').odc.colorize(robust=True, cmap="bone").odc.compress("webp")
)

In [ ]:
100 * xx.odc.geobox.project(xx.odc.geobox.extent ^ xx.odc.geobox.approx.extent).area / (
    xx.shape[0] * xx.shape[1]
)

In [ ]:
%%debug 
xx.reflectance.sel(band=450, method='nearest').odc.reproject(3857)

In [ ]:
xx.odc._state

In [ ]:
sdims = set(xx.odc.spatial_dims)

#coords = dict((k, v) for k, v in src.coords.items() if k not in sdims)
{k: coord for k, coord in xx.reflectance.coords.items() if not sdims.intersection(coord.dims)}

In [ ]:
sdims = set(sdims)
sdims.intersection(('b',))

In [ ]:
display(
    gbox.grid_lines(300) | gbox.extent.exterior | gbox[0, 0].extent.centroid,
    gbox.grid_lines(200, "native"),
)

In [ ]:
(lon[0, 0], lat[0, 0]), gbox.gcps()[:10]

In [ ]:
c_lon, c_lat = footprint.centroid.to_crs(4326).points[0]

# ax = plt.axes(projection=ccrs.PlateCarree())
# ax = plt.axes(projection=ccrs.Projection("epsg:6933"))
# ax = plt.axes(projection=ccrs.ObliqueMercator())
ax = plt.axes(
    # projection=ccrs.NearsidePerspective(central_latitude=c_lat, central_longitude=c_lon)
    projection=ccrs.Projection(gbox.crs)
)

# ax.set_extent([-8, 15, 34, 50], ccrs.Geodetic())
ax.plot(XX, YY, "C1o", transform=ccrs.Geodetic())
ax.plot([c_lon], [c_lat], "C1x", transform=ccrs.Geodetic())
# ax.add_geometries(footprint.geom, ccrs.CRS(footprint.crs.proj), alpha=0.1)
ax.add_geometries(gbox.grid_lines(200, "native").geom, ccrs.CRS(gbox.crs.proj), alpha=1)
ax.coastlines()
ax.gridlines()

In [ ]:
np.asarray(gg.json["coordinates"])

In [ ]:
x0, y0 = gbox.affine * (0, 0)
x1, y1 = gbox.affine * (0, 100)
dx, dy = x1 - x0, y1 - y0
dx, dy, dx / dy, np.arctan2(dy, dx) * 180 / np.pi

In [ ]:
gbox.affine * (0, 0)

In [ ]:
footprint.to_crs("utm").crs

In [ ]:
plt.imshow(pix[:, :, 80], vmin=0)
plt.colorbar()

In [ ]:
plt.imshow(elev)

In [ ]:
footprint.to_crs("utm")

In [ ]:
def emit_to_zarr(doc):
    return to_zarr(ElementTree.fromstring(doc))


def kk(tar_stream):
    for p, doc in tar_stream:
        yield {"url": f"s3://{p}", "zarr": emit_to_zarr(doc)}


tar_stream = tar_doc_stream("emit-dmrpp.tgz")
to_njson(tqdm(kk(tar_stream)), "emit-zarr-meta.njson.gz")

In [ ]:
# geotransfrom: applies to `location/glt_{x,y}` images
#  tx, sx, 0, ...
#  ty, 0, sy

attrs = meta_store[".zattrs"]

kk = [
    "flight_line",
    "time_coverage_start",
    "time_coverage_end",
    #'crosstrack_orientation',
    "spatialResolution",
    #'spatial_ref',
    #'geotransform',
    "day_night_flag",
    #'title'
]
{**{k: attrs[k] for k in kk}, "bbox": tuple(attrs[k] for k in BBOX_KEYS)}

In [ ]:
B = "reflectance"
meta_store[f"{B}/.zarray"], meta_store[f"{B}/.zchunkstore"], meta_store[f"{B}/.zattrs"]

In [ ]:
B = "location/lat"
B = "sensor_band_parameters/wavelengths"
meta_store[f"{B}/.zarray"], meta_store[f"{B}/.zchunkstore"], meta_store[f"{B}/.zattrs"]

In [ ]:
[(B, meta_store[f"{B}/.zchunkstore"].get("0", ":chunked:")) for B in bands]

In [ ]:
# ORBIT = 2329804 # AU
ORBIT = 2315109

urls_all = slurp_lines("emitl2arfl.urls.gz")

nc_rgx = re.compile(
    "s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_.*/EMIT_L2A_RFL_001_\d{8}T\d{6}_\d+_\d{3}.nc$"
)

uu = [u for u in urls_all if nc_rgx.match(u)]
len(uu), [u for u in uu if parse_emit_orbit(u) == ORBIT]

In [ ]:
# meta_store['.zattrs']

In [ ]:
%%bash

. /tmp/earthdata-s3.env

granule=EMIT_L2A_RFL_001_20230531T133036_2315109_002
granule=EMIT_L2A_RFL_001_20230531T133124_2315109_003

aws s3 ls s3://lp-prod-protected/EMITL2ARFL.001/$granule/
#aws s3 cp s3://lp-prod-protected/EMITL2ARFL.001/$granule/$granule.nc.dmrpp .
#aws s3 cp s3://lp-prod-protected/EMITL2ARFL.001/$granule/$granule.cmr.json .
#aws s3 cp s3://lp-prod-protected/EMITL2ARFL.001/$granule/$granule.nc .

#ln -s $granule.nc sample-003.nc
#ln -s $granule.nc.dmrpp sample-003.nc.dmrpp
